In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# 이미지에 들어가는 한글을 제대로 보기 위해 한글 폰트 적용
import platform
font_dict = {
    'Linux': 'Noto Sans CJK KR',
    'Darwin': 'Apple SD Gothic Neo', # macOS
    'Windows': 'Malgun Gothic' # MS-Windows
}
try:
    mpl.rc('font', family=font_dict[platform.system()])
except:
    pass
mpl.rc('axes', unicode_minus=False) # mpl.rcParams['axes.unicode_minus'] = False

%matplotlib inline

# [All about 따릉이 EDA, 1편] 대여소 살펴보기 by 흠시

**출처:** https://dailyheumsi.tistory.com/86, https://github.com/heumsi/Seoul-Public-bicycles-EDA

**데이터:** 서울특별시 공공자전거 대여소 정보 @[서울 열린데이터 광장](https://data.seoul.go.kr)
  - 대여소의 이름, 관리번호, 위치정보와, 거치대수 정보를 제공
  - https://data.seoul.go.kr/dataList/OA-13252/F/1/datasetView.do
  - `서울특별시 공공자전거 대여소 정보(19.12.9).xlsx` (용량 0.3MB, 수정일 2019.12.10)

파일을 다루는 방법은 운영체제들마다 조금씩 다르다. (특히, 마이크로소프트 윈도)
파이썬에는 운영체제에 상관없이 파일의 경로(path)를 다루는 객체지향형 패키지 pathlib이 기본으로 들어있다.
자세한 것은 https://docs.python.org/3/library/pathlib.html 를 참조하자.

In [ ]:
from pathlib import Path

데이터_폴더 = Path('../data')

# Path 클래스를 이용하면 아래와 같이 백슬래시(`/`) 를 이용해 바로 파일을 붙여 쓸 수 있다.
공공자전거_대여소_정보 = 데이터_폴더 / '서울특별시 공공자전거 대여소 정보(19.12.9).xlsx'
공공자전거_대여소_정보

In [ ]:
rental = pd.read_excel(공공자전거_대여소_정보)
rental.head()

In [ ]:
rental.tail()

In [ ]:
# 마지막 줄(1540)은 합계 정보이므로 읽어들이지 않는다.

rental = pd.read_excel(공공자전거_대여소_정보, skipfooter=1)
rental.tail()

In [ ]:
# pandas.read_excel() 함수에 skipfooter 옵션이 있는 지 어떻게 아는가?

pd.read_excel?

In [ ]:
# 읽어들인 데이터의 정보를 파악하려면...

rental.info()

위의 `rental.info()` 결과를 보면 모든 컬럼이 1540개로 동일하다.
즉, 결측치(missing value)가 없다는 이야기다.

결측치를 시각화하는 도구로 [missingno](https://github.com/ResidentMario/missingno) 패키지가 편리하다.

> Messy datasets? Missing values? `missingno` provides a small toolset of flexible and easy-to-use missing data visualizations and utilities that allows you to get a quick visual summary of the completeness (or lack thereof) of your dataset.

missingno 패키지는 `pip install missingno` 명령으로 설치할 수 있다.

In [ ]:
import missingno as msno

msno.matrix(rental);

---
## 1. 따릉이 대여소 위치 무작정 시각화 해보기

> [흠시] 일단 정말, 무작정 시각화 해보자.

지도를 이용한 시각화 도구로 [folium](https://python-visualization.github.io/folium/) 패키지가 편리하다.

> `folium` builds on the data wrangling strengths of the Python ecosystem and the mapping strengths of the `leaflet.js` library. Manipulate your data in Python, then visualize it in on a Leaflet map via `folium`.

**Reference:** [Leaflet - a JavaScript library for interactive maps](https://leafletjs.com)

folium 패키지도 `pip install folium` 명령으로 설치하자.

In [ ]:
# 읽어들인 데이터프레임에서 위도, 경도 컬럼만 선택한다.

rental_loc = rental[['위도', '경도']]
rental_loc

In [ ]:
# 각 컬럼의 평균을 구하자.
# DataFrame.mean()에 아무 옵션도 주지 않으면 로우(인덱스)를 따라 (axis=0) 각 컬럼의 평균을 구한다.
# DataFrame.mean(axis=1)과 같이 옵션을 주면 컬럼을 따라 각 로우의 평균을 구한다.

rental_loc_center = rental_loc.mean()
rental_loc_center

#### Create a Map with Folium and Leaflet.js

```python
class folium.folium.Map(location=None, width='100%', height='100%', left='0%', top='0%', position='relative', tiles='OpenStreetMap', attr=None, min_zoom=0, max_zoom=18, zoom_start=10, min_lat=- 90, max_lat=90, min_lon=- 180, max_lon=180, max_bounds=False, crs='EPSG3857', control_scale=False, prefer_canvas=False, no_touch=False, disable_3d=False, png_enabled=False, zoom_control=True, **kwargs)
```

Generate a base map of given `width` and `height` with either default tilesets or a custom tileset URL.
The following tilesets are built-in to Folium. Pass any of the following to the `tiles` keyword:

  - `OpenStreetMap` (한글지도)
  - `CartoDB positron` or `CartoDB dark_matter`
  - `Stamen Terrain`, `Stamen Toner`, or `Stamen Watercolor`
  - `Mapbox Bright` or `Mapbox Control Room` (Limited levels of zoom for free tiles)
  - `Cloudmade` or `Mapbox` (Must pass API key)

In [ ]:
# 먼저 지도를 그려보자. 이때 location에는 지도의 중심이 될 위도와 경도를 넣어준다.

import folium

bike_map = folium.Map(
    location=rental_loc_center,
    zoom_start=10.8,
    tiles='CartoDB positron'
)
bike_map

#### A circle of a fixed size with radius specified in pixels

```python
class folium.vector_layers.CircleMarker(location, radius=10, popup=None, tooltip=None, **kwargs)
```

**Parameters:**

  - `location` (tuple[float, float]) – Latitude and Longitude pair (Northing, Easting)
  - `radius` (float, default 10) – Radius of the circle marker, in pixels.
  - `popup` (str or `folium.Popup`, default `None`) – Input text or visualization for object displayed when clicking.
  - `tooltip` (str or `folium.Tooltip`, default `None`) – Display a text when hovering over the object.
  - `**kwargs` – Other valid (possibly inherited) options. See: https://leafletjs.com/reference-1.6.0.html#circlemarker

In [ ]:
# 아래에서 각 로우의 인덱스 정보는 사용하지 않는다는 의미로 언더스코어(`_`)로 표시했다.

for _, loc in rental_loc.iterrows():
    folium.CircleMarker(loc,
        radius=.1,
        fill=True,
    ).add_to(bike_map)
bike_map

> [흠시] 곳곳에 설치된 따릉이 대여소들  
뭐.. 여기저기 많은 것 같다.

> [흠시] 히트맵으로 대여소들의 밀집도를 살펴보면,

#### Create a Heatmap layer

```python
class folium.plugins.HeatMap(data, name=None, min_opacity=0.5, max_zoom=18, max_val=1.0, radius=25, blur=15, gradient=None, overlay=True, control=True, show=True, **kwargs)
```

**Parameters:**

  - `data` (list of points of the form `[lat, lng]` or `[lat, lng, weight]`) – The points you want to plot. You can also provide a `numpy.array` of shape `(n,2)` or `(n,3)`.
  - `name` (string, default `None`) – The name of the Layer, as it will appear in LayerControls.
  - `min_opacity` (default 1.) – The minimum opacity the heat will start at.
  - `max_zoom` (default 18) – Zoom level where the points reach maximum intensity (as intensity scales with zoom), equals maxZoom of the map by default
  - `max_val` (float, default 1.) – Maximum point intensity
  - `radius` (int, default 25) – Radius of each “point” of the heatmap
  - `blur` (int, default 15) – Amount of blur
  - `gradient` (dict, default `None`) – Color gradient config. e.g. {0.4: ‘blue’, 0.65: ‘lime’, 1: ‘red’}
  - `overlay` (bool, default `True`) – Adds the layer as an optional overlay (True) or the base layer (False).
  - `control` (bool, default `True`) – Whether the Layer will be included in LayerControls.
  - `show` (bool, default `True`) – Whether the layer will be shown on opening (only for overlays).

#### Adds a fullscreen button to your map

```python
class folium.plugins.Fullscreen(position='topleft', title='Full Screen', title_cancel='Exit Full Screen', force_separate_button=False, **kwargs)
```

Parameters

  - `position` (str) – change the position of the button can be: ‘topleft’, ‘topright’, ‘bottomright’ or ‘bottomleft’ default: ‘topleft’
  - `title` (str) – change the title of the button, default: ‘Full Screen’
  - `title_cancel` (str) – change the title of the button when fullscreen is on, default: ‘Exit Full Screen’
  - `force_separate_button` (bool, default `False`) – force seperate button to detach from zoom buttons

In [ ]:
import folium.plugins

bike_map = folium.Map(
   location=rental_loc_center,
   zoom_start=10.8,
   tiles='CartoDB positron'
)
# 지도에 히트맵을 추가
folium.plugins.HeatMap(rental_loc,
    radius=10,
    blur=5
).add_to(bike_map)
# 지도에 전체화면 버튼을 추가
folium.plugins.Fullscreen(
    position='topright'
).add_to(bike_map)
bike_map

> [흠시] 역시 아무래도 **서울 한 가운데 지점이 눈에 띄게 밀집도가 높은 것**을 알 수 있다.

---
## 2. 지역구 중심으로 살펴보기

> [흠시] 이번에는 지역구를 중심으로 표현해보자.

### 2.1. 대여소 개수 살펴보기

각 지역구의 구획을 지도 위에 표시해 보자. 이러한 정보는 GeoJSON 포맷으로 넣어주어야 한다.

> **GeoJSON** is an open standard format designed for representing simple geographical features, along with their non-spatial attributes. It is based on the JSON format. (from [Wikipedia](https://en.wikipedia.org/wiki/GeoJSON))

**Reference:**
  1. `seoul_municipalities_geo_simple.json` (193KB) from [Seoul-Public-bicycles-EDA (github:heumsi)](https://github.com/heumsi/Seoul-Public-bicycles-EDA)
  2. `skorea-municipalities-geo.json` (16.1MB) from [South Korea Maps (github:southkorea)](https://github.com/southkorea/southkorea-maps)

In [ ]:
import json, gzip

geo_path = 데이터_폴더 / 'seoul_municipalities_geo_simple.json'
with geo_path.open(encoding='utf-8') as fp:
    geo_str = json.load(fp)

# geo_path = 데이터_폴더 / 'skorea-municipalities-geo.json.gz'
# with gzip.GzipFile(geo_path, 'r') as fp:
#     geo_str = json.load(fp)

geo_str['features'][0]['properties']

#### Apply a GeoJSON overlay to the map

```python
class folium.features.Choropleth(geo_data, data=None, columns=None, key_on=None, bins=6, fill_color=None, nan_fill_color='black', fill_opacity=0.6, nan_fill_opacity=None, line_color='black', line_weight=1, line_opacity=1, name=None, legend_name='', overlay=True, control=True, show=True, topojson=None, smooth_factor=None, highlight=None, **kwargs)
```

Plot a GeoJSON overlay on the base map. There is no requirement to bind data (passing just a GeoJSON plots a single-color overlay), but there is a data binding option to map your columnar data to different feature objects with a color scale.

If `data` is passed as a Pandas DataFrame, the `columns` and `key-on` keywords must be included, the first to indicate which DataFrame columns to use, the second to indicate the layer in the GeoJSON on which to key the data. The `columns` keyword does not need to be passed for a Pandas series.

Colors are generated from color brewer (http://colorbrewer2.org) sequential palettes. By default, linear binning is used between the min and the max of the values. Custom binning can be achieved with the `bins` parameter.

TopoJSONs can be passed as `geo_data`, but the `topojson` keyword must also be passed with the reference to the topojson objects to convert. See the topojson.feature method in the TopoJSON API reference: https://github.com/topojson/topojson/wiki/API-Reference

**Parameters:**

  - `geo_data` (string/object) – URL, file path, or data (json, dict, geopandas, etc) to your GeoJSON geometries
  - `data` (Pandas DataFrame or Series, default None) – Data to bind to the GeoJSON.
  - `columns` (dict or tuple, default None) – If the data is a Pandas DataFrame, the columns of data to be bound. Must pass column 1 as the key, and column 2 the values.
  - `key_on` (string, default None) – Variable in the geo_data GeoJSON file to bind the data to. Must start with ‘feature’ and be in JavaScript objection notation. Ex: ‘feature.id’ or ‘feature.properties.statename’.
  - `bins` (int or sequence of scalars or str, default 6) – If bins is an int, it defines the number of equal-width bins between the min and the max of the values. If bins is a sequence, it directly defines the bin edges. For more information on this parameter, have a look at numpy.histogram function.
  - `fill_color` (string, optional) – Area fill color, defaults to blue. Can pass a hex code, color name, or if you are binding data, one of the following color brewer palettes: ‘BuGn’, ‘BuPu’, ‘GnBu’, ‘OrRd’, ‘PuBu’, ‘PuBuGn’, ‘PuRd’, ‘RdPu’, ‘YlGn’, ‘YlGnBu’, ‘YlOrBr’, and ‘YlOrRd’.
  - `nan_fill_color` (string, default 'black') – Area fill color for nan or missing values. Can pass a hex code, color name.
  - `fill_opacity` (float, default 0.6) – Area fill opacity, range 0-1.
  - `nan_fill_opacity` (float, default fill_opacity) – Area fill opacity for nan or missing values, range 0-1.
  - `line_color` (string, default 'black') – GeoJSON geopath line color.
  - `line_weight` (int, default 1) – GeoJSON geopath line weight.
  - `line_opacity` (float, default 1) – GeoJSON geopath line opacity, range 0-1.
  - `legend_name` (string, default empty string) – Title for data legend.
  - `topojson` (string, default None) – If using a TopoJSON, passing “objects.yourfeature” to the topojson keyword argument will enable conversion to GeoJSON.
  - `smooth_factor` (float, default None) – How much to simplify the polyline on each zoom level. More means better performance and smoother look, and less means more accurate representation. Leaflet defaults to 1.0.
  - `highlight` (boolean, default False) – Enable highlight functionality when hovering over a GeoJSON area.
  - `name` (string, optional) – The name of the layer, as it will appear in LayerControls
  - `overlay` (bool, default True) – Adds the layer as an optional overlay (True) or the base layer (False).
  - `control` (bool, default True) – Whether the Layer will be included in LayerControls.
  - `show` (bool, default True) – Whether the layer will be shown on opening (only for overlays).

Returns GeoJSON data layer in `obj.template_vars`

In [ ]:
bike_map = folium.Map(
    location=rental_loc_center,
    zoom_start=11,
    tiles='CartoDB positron'
)
# 지도에 지역구 구획을 표시
folium.Choropleth(geo_str).add_to(bike_map)
bike_map

In [ ]:
rental['대여소_구'].value_counts()

In [ ]:
# rental['대여소_구'].value_counts()

have_bikes = (rental
    .groupby('대여소_구')
    .size() # returns Series
    .sort_values(ascending=False)
)
have_bikes

In [ ]:
bike_map = folium.Map(
    location=rental_loc_center,
    zoom_start=11,
    tiles='CartoDB positron'
)
folium.Choropleth(geo_str,
    data=have_bikes,
    key_on='feature.properties.SIG_KOR_NM', # seoul_municipalities_geo_simple.json
#    key_on='feature.properties.NL_NAME_2', # skorea-municipalities-geo.json.gz
    fill_color='Blues'
).add_to(bike_map)
bike_map

> [흠시] 지역별 대여소 개수. 색이 진할수록 해당 지역에 대여소 개수가 많다는 뜻이다.  
**강남 3구(서초, 송파, 강남)와, 한강 서쪽 지역(마포, 강서, 영등포)**이 대여소가 많은 것을 한 눈에 알 수 있다.

> [흠시] 조금 더 수치적으로 확인해보면,

In [ ]:
# https://matplotlib.org/api/_as_gen/matplotlib.patches.Rectangle.html
# https://matplotlib.org/api/transformations.html#matplotlib.transforms.Bbox

ax = have_bikes.plot.bar(
    rot=0,
    xlabel="",
    title="지역구별 대여소 개수",
    figsize=(15, 4)
)
ax.set_frame_on(False)
for p in ax.patches: # p is an instance of matplotlib.patches.Rectangle
    x, y, width, height = p.get_bbox().bounds
    ax.annotate(f"{height:.0f}", (x+width/2, height), ha='center')

> [흠시] 가장 많은 곳(강남구, 95개) 과 가장 적은 곳(도봉구, 31개)의 차이가 약 **3배 가량** 나는 것을 알 수 있다.

### 2.2 대여소당 거치대 수 살펴보기

> [흠시] 하나의 따릉이 대여소에는 따릉이를 거치해놓을 수 있는 여러 개의 거치대가 있는데, 이 거치대 수에 대해서도 한번 살펴보자.  
먼저, 서울 시내 전체에서, 거치대 수의 분포는 어떨까?

In [ ]:
# https://seaborn.pydata.org/generated/seaborn.histplot.html

_, ax = plt.subplots(figsize=(15, 4))
sns.histplot(rental['거치대수'], stat='density', kde=True, linewidth=0, ax=ax)
ax.set_xticks(range(0, 41))
ax.set_title("거치대수 분포")
ax.set_frame_on(False)

> [흠시] 일반적으로 **10개, 조금 더 많은 곳은 15개, 20개**가 대표적이다.  
간혹가다, 30개, 40개인 곳도 보이는데, 이런 곳은 아무래도 대형 대여소인듯 하다.

> [흠시] 이제 다시, 지역별로 대여소당 평균 거치대 수를 살펴보면,

In [ ]:
bike_rental = (rental['거치대수']
    .groupby(rental['대여소_구'])
    .sum()
    .divide(have_bikes)
    .sort_values(ascending=False)
)
bike_rental

In [ ]:
bike_map = folium.Map(
    location=rental_loc_center,
    zoom_start=10.8,
    tiles='CartoDB positron'
)
folium.Choropleth(geo_str,
    data=bike_rental,
    key_on='feature.properties.SIG_KOR_NM', 
    fill_color='Blues'
).add_to(bike_map)
bike_map

> [흠시] 지역별 대여소의 평균 거치대수 개수. 색이 진할수록 값이 크다.  
마포, 영등포, 서초의 경우, 대여소 수도 많았지만, 평균 거치대수도 많다. 즉 따릉이가 제일 많이 비치되어있는 지역들이다.  
용산의 경우, 대여소 수는 적었지만, 평균 거치대수는 많다. 즉, 한 대여소에 한 번에 많은 따릉이가 비치되어있다.

> [흠시] 좀 더 자세히 보기 위해, 수치값을 보면

In [ ]:
ax = bike_rental.plot.bar(
    rot=0,
    xlabel="",
    title="지역구별 평균 거치대수",
    figsize=(15, 4)
)
ax.set_frame_on(False)
for p in ax.patches: 
    x, y, width, height = p.get_bbox().bounds 
    ax.annotate(f"{height:.1f}", (x+width/2, height), ha='center')

> [흠시] 지역별 평균 거치대수는 10~15대 사이로, **엄청 큰 차이를 보이지는 않는다.**

### 2.3. 가장 가까운 대여소간 평균 거리는 얼마나 다를까?

> [흠시] 하나의 대여소와 가장 가까운 대여소의 거리들의 평균을 구해보자.  
즉, 대여소가 모여있다면, 평균 거리는 작을 것이고, 흩어져있다면, 평균 거리는 클 것이다.  

> [흠시] 먼저, 각 지역별로, 가장 가까운 대여소간 거리의 분포를 살펴보면.

In [ ]:
import math

def haversine(p1, p2):
    """
    Calculate the great circle distance between two points on the earth (specified in radians)

    Reference: https://en.wikipedia.org/wiki/Haversine_formula
    """
    def hav(theta):
        return 0.5 * (1. - math.cos(theta))

    (lat1, lon1), (lat2, lon2) = p1, p2
    # haversine formula 
    h = hav(lat2 - lat1) + math.cos(lat1) * math.cos(lat2) * hav(lon2 - lon1)
    # mean earth radius (https://en.wikipedia.org/wiki/Earth_radius)
    r = 6.371e6
    return 2. * r * math.asin(math.sqrt(h))

In [ ]:
%%time

def _min_haversine(df):
    n = len(df)
    a = np.radians(df[['위도', '경도']].to_numpy())
    uf = np.vectorize(lambda i, j: haversine(a[i], a[j]) if i != j else np.inf)
    indices = np.indices((n, n)).reshape((2, -1))
    return uf(*indices).reshape(n, -1).min(axis=1)

dict_dists = {
    name: _min_haversine(df) for name, df in rental.groupby('대여소_구')
}

In [ ]:
fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(15, 10), sharex=True, sharey=True)
axes = axes.ravel()

for i, (name, dists) in enumerate(dict_dists.items()):
    sns.histplot(dists, bins=10, stat='density', kde=True, linewidth=0, ax=axes[i])
    axes[i].set_xlim((0, 1000))
    axes[i].set_title(name)
    axes[i].axvline(dists.mean(), color='green')

fig.suptitle("가까운 대여소와의 거리 분포", y=1.01)
fig.tight_layout()

> [흠시] 초록색 선은 해당 지역의 평균 값(평균 거리) 이다.  
가장 가까운 대여소간의 거리는 대부분 **200~400m** 안이다.  
관악구의 경우, 봉우리가 2개 있는 분포가 눈에 띈다. 밀집된 대여소가 두 군데로 보여진다. 이는 직접 확인해봐야겠다.  

> [흠시] 잠깐 관악구의 대여소 분포를 지도에서 살펴보면,

In [ ]:
# rental.query("대여소_구 == '관악구'")[['위도', '경도']]
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html

gwanak = rental[rental['대여소_구'] == '관악구'][['위도', '경도']]
gwanak.info()

In [ ]:
gwanak_center = gwanak.mean()
gwanak_center

In [ ]:
bike_map = folium.Map(
    location=gwanak_center,
    zoom_start=14
)
for _, location in gwanak.iterrows():
    folium.CircleMarker(location,
        radius=5,
        fill=True,
    ).add_to(bike_map)
folium.plugins.HeatMap(gwanak).add_to(bike_map)
bike_map

> [흠시] 예상했던 대로, 신림역과 서울대입구역 근방으로 대여소들이 밀집되어 있음을 확인할 수 있다.

> [흠시] 다시 돌아가, 지역별로 가까운 대여소 평균 거리를 살펴보자.  
이번에는 수치값도 나름 중요하므로, 시각화된 지도와 막대 그래프를 같이본 뒤 해석해보자.

In [ ]:
dist_means = pd.Series(
    {name: dists.mean() for name, dists in dict_dists.items()}
).sort_values()
dist_means

In [ ]:
bike_map = folium.Map(
    location=rental_loc_center,
    zoom_start=10.8,
    tiles='CartoDB positron'
)
folium.Choropleth(geo_str,
    data=dist_means,
    key_on='feature.properties.SIG_KOR_NM', 
    fill_color='Blues'
).add_to(bike_map)
bike_map

> [흠시] 색이 옅을수록 평균거리가 짧은 것이다.

In [ ]:
ax = dist_means.plot.bar(
    rot=0,
    legend=False,
    title="지역구별 가까운 대여소간 거리 평균(m)",
    figsize=(15, 4)
)
ax.set_frame_on(False)
for p in ax.patches: 
    x, y, width, height = p.get_bbox().bounds 
    ax.annotate(f"{height:.0f}", (x+width/2, height), ha='center')

> [흠시] 지역별 대여소간 평균거리는 240~350m 안쪽이다.  
대여소, 거치대수가 많던 마포구가, 이번에도, 대여소간 평균 거리가 제일 작았다. 이쯤 되면, 마포에 좀 주목할 필요가 있어보인다.    
서울 한가운데에 있는 종로와 성북이 그 다음으로 작았다. 대여소가 밀집되어있는 대표적인 지역들이다.  
강남 3구 중, 유독 송파만 거리가 짧았다. 이후에, 좀 더 구체적으로 살펴볼만할 듯 하다.  
도봉, 동대문, 양천은 평균거리가 유독 컸다. 이유가 뭘까? 이 역시 이후에 좀 더 구체적으로 살펴봐야겠다.  

---
## 정리 

1. 서울에 전반적으로 대여소가 잘 설치되어있다. 그 중, 특히 **서울 가운데 지역에 조금 더 밀집**이 되어있다.
2. 대여소의 개수는 **강남3구와 강서지역(강서, 마포, 영등포)**에 유독 많았고, 제일 적은 지역과 가장 많은 지역의 차이는 **3배**정도 난다.
3. 한 대여소당 평균 거치대 수는 **10~15개**로, 영등포와 용산, 서초가 가장 많았다.
4. 지역별 대여소간 가장 가까운 평균거리는 **240~350m로, 마포구(241m)가 제일 작았다.** 제일 작은 지역과 큰 지역간의 차이는 100m 가량 난다.